In [74]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import minimize

# Modelo de regresión simple

Sea $ϵ_1,..., ϵ_n $ iid N~[0; $σ^2$]. Considere que cada $ϵ_1$ puede ser escrito como una función de datos observados para $x_i$ y $y_i$. Es decir, $ϵ_1$ = $y_i-α−βx_ i$

### 1. Escriba la funcion log-likelihood.

La función $ = y_i=α+βx_ i+ϵ_1$ puede ser escrita en su forma matricial:
$$ Y= β X + ϵ $$
tal que
$$ Y= [yi]  ; β= \begin{bmatrix} \alpha \\ \beta \end{bmatrix}; X= \begin{bmatrix} 1 & x_i \end{bmatrix}; ϵ=[ϵ_i]  $$


La función de densidad condicional para una observación es:
$$
f(Y \mid X; , \beta, \sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left( -\frac{(Y-X \beta)'(Y-X \beta)}{2\sigma^2} \right)
$$

Entonces la funcion de log-likelihood es:
$$
\log L( \beta, \sigma^2) = \sum_{i=1}^n \log f(Y \mid X; \beta, \sigma^2)
$$

$$
= -\frac{n}{2} \log(2\pi) - \frac{n}{2} \log(\sigma^2) - \frac{1}{2\sigma^2} \sum_{i=1}^n (Y-X \beta)'(Y-X \beta)
$$

### 2. Encuentre el estimador de maxima verosimilitud para α, β, $σ^2$

Primero maximizamos la funcion log-likelihood con respecto a β para encontrar β^:
$$
\frac{\partial \log L}{\partial \beta} = \frac{1}{\sigma^2} X' (Y-X\beta)=0
$$
Y encontramos que β^ es: 
$$
\hat{\beta} =  (X' X)^{-1} (X'Y)
$$

Tal que 
$$ \hat{\beta}= \begin{bmatrix} \hat{\alpha} \\ \hat{\beta} \end{bmatrix}

Finalmente derivamos con respecto a $σ^2$
$$

\frac{dlog L}{d\sigma^2}= -\frac{n}{2 \sigma^2} + \frac{1}{2 \sigma^4} (Y-X \beta)'(Y-X \beta)
$$
y obtenemos que:
$$
\hat{\sigma^2}= \frac{1}{n} (Y-X \hat{\beta})'(Y-X \hat{\beta})
$$
dado que:
$$
RSS= (Y-X \hat{\beta})'(Y-X \hat{\beta})= ϵ'ϵ
$$
Entonces:
$$
\hat{\sigma^2}= \frac{RSS}{n} = \frac{ϵ'ϵ}{n}

### 3. Encuentre el limite inferior Cramer-Rao.

$$S(\theta) = \begin{bmatrix} \frac{1}{\sigma^2} X' (Y-X \ \beta) \\ -\frac{n}{2 \sigma^2} + \frac{1}{2 \sigma^4} (Y-X \beta)'(Y-X \beta) \end{bmatrix} $$
$\quad$
$$ I(\theta) = E[S(\theta) S(\theta)'] $$
$\quad$
$$ I(\theta) = E[\begin{bmatrix} \frac{1}{\sigma^2} X'X & \frac{1}{\sigma^2} X' (Y-X\beta ) \\ \frac{1}{\sigma^2} X' (Y-X\beta ) & \frac{n}{2 \sigma^4} - \frac{1}{\sigma^6} (Y-X\beta )' (Y-X\beta )  \end{bmatrix}] $$
$\quad$
$$ I(\theta) = \begin{bmatrix} \frac{1}{\sigma^2} X'X & 0 \\ 0' & \frac{n}{2 \sigma^4} \end{bmatrix} $$
$$ Cota \quad de \quad Cramer-Rao= I(\theta)^{-1} = \begin{bmatrix} \sigma^2 (X'X)^{-1} & 0 \\ 0' & \frac{2 \sigma^4}{n} \end{bmatrix}  $$

### 4. Escriba el Likelihood Ratio test para H0 : β = 0

Sea la funcion de verosimilitud sin restriccion:
$$ L_u= (\frac{2 \pi}{n})^{-n/2} * exp(-\frac{n}{2}) * (RSS_u)^{-n/2} $$
y la funcion con restriccion
$$ L_R= (\frac{2 \pi}{n})^{-n/2} * exp(-\frac{n}{2}) * (RSS_R)^{-n/2} $$
El ratio sera:
$$ \lambda = \frac{L_u}{L_R} = (\frac{RSS_u}{RSS_R})^{-n/2} $$


### 5. Escriba un programa en Python que le permita estimar por ML la siguiente expresion:
$$
\text{wage}_i = \beta_1 \cdot \text{education}_i + \beta_2 \cdot \text{experience}_i + \beta_3 \cdot \text{exp2}_i + \beta_4 + \varepsilon_i
$$
Para ello, use la base de datos de la Encuesta de Población Actual (CPS), provista en HMW

In [75]:
url=r"C:\Users\Camil\Documents\GitHub\ECOP2037_CE\hmw2\cps09mar.csv"
df = pd.read_csv(url)

In [76]:
df['wage']=df["earnings"]/df["week"]/df["hours"]
df['experience']=df['age']-df['education']-6
df['exp2']=df['experience']**2
y = df["wage"].values
X = df[["education", "experience", "exp2"]].copy()
X = X[[ "education", "experience", "exp2"]].values
X = np.column_stack((X, np.ones(len(X))))  # Agrega la constante (beta_4)


In [79]:
def neg_log_likelihood(params):
    beta = params[:-1]
    log_sigma = params[-1]
    sigma = np.exp(log_sigma)  # Forzamos sigma > 0

    xb = X @ beta
    ll = norm.logpdf(y, loc=xb, scale=sigma)
    
    if np.any(np.isnan(ll)):
        return np.inf  # penaliza si hay problemas
    return -np.sum(ll)

# Valores iniciales: [0, 0, 0, 0, log(1)]
init_params = np.zeros(X.shape[1] + 1)

# Optimización
result = minimize(neg_log_likelihood, init_params, method='BFGS')

# Resultados
if result.success:
    est_params = result.x
    beta = est_params[:-1]
    sigma = np.exp(est_params[-1])  # Reconvertimos sigma
    print("Estimación exitosa.")
    print("Coeficientes beta:", beta)
    print("Sigma (desvío estándar del error):", sigma)
else:
    print("Fallo en la optimización:", result.message)

Fallo en la optimización: Desired error not necessarily achieved due to precision loss.
